In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import os
import pickle

from collections import Counter

# import pandas as pd
from sklearn.metrics import classification_report

import numpy as np
import torch
import torch.nn as nn

import transformers
from transformers import Trainer
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorWithPadding

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

In [3]:
dataset_mlc = torch.load(os.path.join("/notebooks/MTL_tasks/data", 'pe_dataset_for_MLC_w_combined.pt'))

In [4]:
dataset_mlc

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'mc', 'cl', 'prem', 'link_present', 'support', 'attack'],
        num_rows: 3770
    })
    test: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_ComponentType', 'relation_SupportAtt

In [5]:
# what do we need: 'text', 'structural_fts_as_text', 'structural_fts_as_text_combined' 'mc', 'cl', 'prem', 'link'
# that's enough

In [6]:
cols_not_needed = ['essay_nr', 'component_id', 'label_and_comp_idxs', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para']

In [7]:
dataset_mlc = dataset_mlc.remove_columns(cols_not_needed)

In [8]:
dataset_mlc

DatasetDict({
    train: Dataset({
        features: ['text', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'mc', 'cl', 'prem', 'link_present', 'support', 'attack'],
        num_rows: 3770
    })
    test: Dataset({
        features: ['text', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'mc', 'cl', 'prem', 'link_present', 'support', 'attack'],
        num_rows: 1260
    })
    validation: Dataset({
        features: ['text', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'mc', 'cl', 'prem', 'link_present', 'support', 'attack'],
        num_rows: 943
    })
})

In [9]:
dataset_mlc = dataset_mlc.remove_columns(['mc', 'cl', 'prem'])

In [10]:
dataset_mlc

DatasetDict({
    train: Dataset({
        features: ['text', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'link_present', 'support', 'attack'],
        num_rows: 3770
    })
    test: Dataset({
        features: ['text', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'link_present', 'support', 'attack'],
        num_rows: 1260
    })
    validation: Dataset({
        features: ['text', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'link_present', 'support', 'attack'],
        num_rows: 943
    })
})

In [11]:
dataset_mlc['train'][100]

{'text': 'we cannot ignore its negative effects',
 'structural_fts_as_text': 'Topic: Advantages and disadvantages of machines instead of human to do the work, Sentence: In my opinion, although using machines have many benefits, we cannot ignore its negative effects., Para Number: 1, First in Para: No, Last in Para: Yes, Is in Introduction: Yes, Is in Conclusion: No',
 'structural_fts_as_text_combined': 'Topic: Advantages and disadvantages of machines instead of human to do the work, Sentence: In my opinion, although using machines have many benefits, we cannot ignore its negative effects., Structural Features: 1, No, Yes, Yes, No',
 'link_present': 1,
 'support': 0,
 'attack': 0}

In [12]:
labels = [label for label in dataset_mlc['train'].features.keys() if label not in ['text', 'structural_fts_as_text', 'structural_fts_as_text_combined']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['link_present', 'support', 'attack']

In [13]:
import transformers
#from transformers import Trainer
from transformers import AutoTokenizer
#from transformers import BertForSequenceClassification
#from transformers import Trainer, TrainingArguments
#from transformers.data.data_collator import DataCollatorWithPadding

In [14]:
#from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    # take a batch of texts
    text = examples["structural_fts_as_text"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=256)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

        encoding["labels"] = labels_matrix.tolist()

    return encoding

In [15]:
encoded_dataset = dataset_mlc.map(preprocess_data, batched=True, remove_columns=dataset_mlc['train'].column_names)

Parameter 'function'=<function preprocess_data at 0x7f5807d89e50> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['attention_mask', 'input_ids', 'labels', 'token_type_ids'])


In [17]:
tokenizer.decode(example['input_ids'])

'[CLS] topic : gender equality at university admission, sentence : therefore, universities follow the requirement of job providers and decide subject suitable for particular gender., para number : 3, first in para : no, last in para : yes, is in introduction : no, is in conclusion : no [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [18]:
example['labels']

[1.0, 0.0, 1.0]

In [19]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['link_present', 'attack']

In [20]:
encoded_dataset.set_format("torch")

In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [25]:
device

device(type='cuda')

In [26]:
batch_size = 16
metric_name = "f1"
nr_epochs = 6
results_folder = "/notebooks/MTL_tasks/results"

In [27]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    
    output_dir=results_folder,          
    
    # params
    num_train_epochs=nr_epochs,               # nb of epochs
    per_device_train_batch_size=batch_size,   # batch size per device during training
    per_device_eval_batch_size=batch_size,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=20,                            # cf. paper Sun et al.
    
    # log
#     logging_dir="/notebooks/Results/bert_sequence_classification/tb_logs",  
#     logging_strategy='steps',
#     logging_steps=20,
    
    # save
    save_strategy='steps',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    # metric_for_best_model='eval_loss' 
    metric_for_best_model=metric_name
    
    
    
#     f"bert-finetuned-pe-mlc",
#     evaluation_strategy = "epoch",
#     save_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     metric_for_best_model=metric_name,
#     #push_to_hub=True,
)

In [28]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_macro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [29]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [30]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  8476,  1024,  5907,  9945,  2012,  2118,  9634,  1010,  6251,
         1024,  3568,  1010,  5534,  3582,  1996,  9095,  1997,  3105, 11670,
         1998,  5630,  3395,  7218,  2005,  3327,  5907,  1012,  1010, 11498,
         2193,  1024,  1017,  1010,  2034,  1999, 11498,  1024,  2053,  1010,
         2197,  1999, 11498,  1024,  2748,  1010,  2003,  1999,  4955,  1024,
         2053,  1010,  2003,  1999,  7091,  1024,  2053,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [31]:
#forward pass
# outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
# outputs

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [33]:
trainer.train()

***** Running training *****
  Num examples = 3770
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1416


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
20,No log,0.632066,0.180803,0.551849,0.127253
40,No log,0.576964,0.354494,0.737655,0.501591
60,No log,0.526048,0.389957,0.737353,0.477200
80,No log,0.497455,0.292237,0.742824,0.550371
100,No log,0.488359,0.299110,0.744490,0.552492
120,No log,0.495714,0.471025,0.786187,0.515376
140,No log,0.447111,0.451490,0.785546,0.533404
160,No log,0.430884,0.464960,0.802625,0.585366
180,No log,0.420063,0.482296,0.817797,0.632025
200,No log,0.411874,0.489968,0.821657,0.641569


***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evaluation *****
  Num examples = 943
  Batch size = 16
***** Running Evalua

TrainOutput(global_step=1416, training_loss=0.32725364340227203, metrics={'train_runtime': 1369.5714, 'train_samples_per_second': 16.516, 'train_steps_per_second': 1.034, 'total_flos': 2975812754503680.0, 'train_loss': 0.32725364340227203, 'epoch': 6.0})

In [34]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 943
  Batch size = 16


{'eval_loss': 0.355195015668869,
 'eval_f1': 0.6807914045391144,
 'eval_roc_auc': 0.8501488643339472,
 'eval_accuracy': 0.6839872746553552,
 'eval_runtime': 9.7796,
 'eval_samples_per_second': 96.425,
 'eval_steps_per_second': 6.033,
 'epoch': 6.0}

### inference on the test set

In [35]:
dataset_mlc['test'][500]

{'text': 'the individual should finance his or her education',
 'structural_fts_as_text': 'Topic: Should the Government Provide Free College?, Sentence: In short, the individual should finance his or her education because it is a personal choice, Para Number: 5, First in Para: Yes, Last in Para: No, Is in Introduction: No, Is in Conclusion: Yes',
 'structural_fts_as_text_combined': 'Topic: Should the Government Provide Free College?, Sentence: In short, the individual should finance his or her education because it is a personal choice, Structural Features: 5, Yes, No, No, Yes',
 'link_present': 1,
 'support': 0,
 'attack': 0}

In [36]:
text = "Using public transportation has a lot of advantages for the modern society facing a lot of problems: the environmental population, the isolation in life, the depletion of natural resources"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [37]:
logits = outputs.logits
logits.shape

torch.Size([1, 3])

In [38]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['link_present']


In [39]:
from transformers.data.data_collator import DataCollatorWithPadding

In [40]:
test_trainer = Trainer(model, data_collator=DataCollatorWithPadding(tokenizer))
test_raw_preds, test_labels, _ = test_trainer.predict(encoded_dataset["test"])
# test_preds = np.argmax(test_raw_preds, axis=0)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 1260
  Batch size = 8


In [41]:
test_raw_preds.shape

(1260, 3)

In [42]:
test_labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [43]:
sigmoid = torch.nn.Sigmoid()

In [44]:
test_raw_preds[100]

array([-1.4057186,  4.134342 , -3.8712614], dtype=float32)

In [45]:
try_pred = sigmoid(torch.tensor(test_raw_preds[0]))

In [46]:
try_pred

tensor([0.6498, 0.1610, 0.5734])

In [47]:
test_preds = sigmoid(torch.tensor(test_raw_preds))

In [48]:
test_preds

tensor([[0.6498, 0.1610, 0.5734],
        [0.7137, 0.3408, 0.0279],
        [0.6490, 0.1831, 0.6336],
        ...,
        [0.1713, 0.9876, 0.0155],
        [0.1483, 0.9877, 0.0183],
        [0.1404, 0.9867, 0.0186]])

In [49]:
# nice! now do the thresholding

In [50]:
predictions = np.zeros(test_preds.shape)
predictions[np.where(test_preds >= 0.5)] = 1
# turn predicted id's into actual label names
#predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
#print(predicted_labels)

In [51]:
predictions

array([[1., 0., 1.],
       [1., 0., 0.],
       [1., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [52]:
from sklearn.metrics import classification_report

In [53]:
print(classification_report(test_labels, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.707     0.758     0.732       516
           1      0.955     0.952     0.953      1016
           2      0.515     0.374     0.433        91

   micro avg      0.853     0.858     0.855      1623
   macro avg      0.726     0.694     0.706      1623
weighted avg      0.851     0.858     0.854      1623
 samples avg      0.881     0.882     0.865      1623



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## task: do separate classification reports for separate classes:

In [54]:
test_labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [55]:
predictions

array([[1., 0., 1.],
       [1., 0., 0.],
       [1., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [56]:
test_labels.shape

(1260, 3)

In [57]:
predictions.shape

(1260, 3)

In [58]:
# separate test labels into separate task classes

In [59]:
# link class

test_labels[:,0]

array([1., 1., 0., ..., 0., 0., 0.], dtype=float32)

In [60]:
# support/attack class

In [61]:
test_labels[:,1:3]

array([[0., 0.],
       [0., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [63]:
test_labels_link_classes = test_labels[:,0]

In [64]:
test_labels_support_attack = test_labels[:,1:3]

In [65]:
test_labels_link_classes.shape

(1260,)

In [66]:
test_labels_support_attack.shape

(1260, 2)

In [67]:
# separate test predictions into separate task classes

In [69]:
test_predictions_link_classes = predictions[:,0]

In [70]:
test_predictions_support_attack = predictions[:,1:3]

In [72]:
test_predictions_link_classes.shape

(1260,)

In [73]:
test_predictions_support_attack.shape

(1260, 2)

In [74]:
# now do two separate classification reports

In [75]:
# for link classes

print(classification_report(test_labels_link_classes, test_predictions_link_classes, digits=3))

              precision    recall  f1-score   support

         0.0      0.823     0.782     0.802       744
         1.0      0.707     0.758     0.732       516

    accuracy                          0.772      1260
   macro avg      0.765     0.770     0.767      1260
weighted avg      0.776     0.772     0.773      1260



In [76]:
# for support/attack class

print(classification_report(test_labels_support_attack, test_predictions_support_attack, digits=3))

              precision    recall  f1-score   support

           0      0.955     0.952     0.953      1016
           1      0.515     0.374     0.433        91

   micro avg      0.928     0.904     0.916      1107
   macro avg      0.735     0.663     0.693      1107
weighted avg      0.918     0.904     0.910      1107
 samples avg      0.794     0.794     0.794      1107



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
